In [30]:
import sys
sys.path.append("/home/benjamin/RewardCurriculum/")

import gymnasium as gym
import panda_gym
from stable_baselines3.common.vec_env import SubprocVecEnv
import json
import numpy as np

import os
from copy import deepcopy
from utils.configs import get_config
from utils.env_wrappers import make_vec_env, get_env
from argparse import Namespace
import cv2
from IPython.display import HTML

In [31]:
os.chdir("/home/benjamin/RewardCurriculum")
folder_path = "/home/benjamin/RewardCurriculum/results/manual_curriculum/2023-10-05_16-44-27_PandaMultiRewardStackDense-v3_tqc_manualtask_0_reward_weights=[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]_total_timesteps=100000_learning_starts=99_learning_rate=0.0005"

args = Namespace()
args.env_name = json.load(open(os.path.join(folder_path, "config.json"), "r"))["environment"]["env_name"]
args.seed = json.load(open(os.path.join(folder_path, "config.json"), "r"))["seed"]
args.config_path = os.path.join(folder_path, "config_original.json")
args.continue_from = None

remaining_args = ["--environment.wrapper_kwargs.0.reward_threshold", "-0.05"]
# remaining_args.__setattr__[] = "-0.05"
config = get_config(args.config_path, args, remaining_args)

In [32]:
make_env_fn = lambda wrappers, wrapper_kwargs, ignore_keyword="ignore" : get_env(config["environment"]["env_name"], wrappers=wrappers, wrapper_kwargs=wrapper_kwargs, ignore_keyword=ignore_keyword)
env = make_vec_env(make_env_fn, 
                    n_envs=config["environment"]["n_envs"], 
                    env_kwargs={"wrappers": config["environment"]["wrappers"], "wrapper_kwargs": config["environment"]["wrapper_kwargs"]},
                    monitor_kwargs={"allow_early_resets": True},
                    seed=config["seed"], vec_env_cls=SubprocVecEnv)

/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


pybullet build time: Sep 15 2023 11:13:12


In [33]:
# learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)
learner = config["learner_class"].load(os.path.join(folder_path, "final.zip"), env=env)

In [34]:
def record_video(video_output_path, env, learner, task=None):
    if task is None:
        task = np.zeros((1, learner.scheduler.reward_dim))
        task[:, 1] = 1
    obs = env.reset()

    terminated = False
    render_imgs = []
    while not terminated:
        # obs = obs.reshape(1, -1)
        act = learner.predict(obs, task=task)[0]
        
        obs, reward, terminated, info = env.step(act)
        # print(reward)
        
        render_imgs.append(env.render())
        
    height, width, _ = render_imgs[0].shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(video_output_path, fourcc=fourcc, fps=10, frameSize=(width, height))

    for frame in render_imgs:
        out.write(frame)
        
    out.release()
    
    os.system(f"ffmpeg -hide_banner -loglevel error -i '{video_output_path}' -vcodec libx264 video.mp4")
    os.system(f"mv video.mp4 '{video_output_path}'")

In [40]:
video_path = os.path.join(folder_path, "video.mp4")
record_video(video_path, env, learner)
HTML(f'<video alt="test" controls><source src="{video_path}" type="video/mp4"></video>')